In [1]:
%matplotlib inline

In [6]:
#export
from export.nb_01 import *
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import matplotlib.pyplot as plt
import os
from collections import OrderedDict
from typing import Tuple
from pathlib import Path

pd.pandas.set_option('display.max_columns', None)

In [7]:
#export
Path.ls= lambda x: [i.name for i in os.scandir()]

In [8]:
path_tabdata=Path(r'D:\datasets\new_datasets')
path_tabdata.ls()

['.ipynb_checkpoints',
 '00_testing.ipynb',
 '01_nn.ipynb',
 '02b_tabular.ipynb',
 '02_callbacks.ipynb',
 '02_df.ipynb',
 '03_dataloader.ipynb',
 '04_transforms.ipynb',
 'callback_dev.ipynb',
 'export',
 'notebook2script.py']

In [9]:
path_house=path_tabdata/'sberbank-russian-housing-market'
path_house.ls()

['.ipynb_checkpoints',
 '00_testing.ipynb',
 '01_nn.ipynb',
 '02b_tabular.ipynb',
 '02_callbacks.ipynb',
 '02_df.ipynb',
 '03_dataloader.ipynb',
 '04_transforms.ipynb',
 'callback_dev.ipynb',
 'export',
 'notebook2script.py']

In [10]:
path_house_train=path_house/'train.csv'
path_house_test=path_house/'test.csv'

In [11]:
df=pd.read_csv(path_house_train, index_col='id', parse_dates=['timestamp'], infer_datetime_format=True)
df.head()

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

In [12]:
#export
def get_sample(df: pd.DataFrame, n:int)-> pd.DataFrame:
    idxs= sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

In [13]:
#export
def train_cats(df:pd.DataFrame):
    for col_name, col in df.items():
        if is_string_dtype(col):
            df[col_name]=col.astype('category').cat.as_ordered()

In [14]:
#export
def apply_cats(df:pd.DataFrame, df_train:pd.DataFrame):
    for col_name, col in df.items():
        if (col_name in df_train.columns) and (df_train[col_name].dtype.name=='category'):
            df[col_name]=pd.Categorical(col, categories=df_train[col_name].cat.categories, ordered=True)

In [15]:
#export
def fill_missing(df:pd.DataFrame, col_name: str, na_dict:dict=None):
    na_dict=() if na_dict is None else na_dict
    col=df[col_name]
    if is_numeric_dtype(col) and (pd.isnull(col).sum() or (col_name in na_dict)):
        df[col_name+'_na']=pd.isnull(col)
        filler=na_dict[col_name] if col_name in na_dict else col.median()
        df[col_name]=col.fillna(filler)
        na_dict[col_name]=filler
    return na_dict

In [16]:
#export
def numericalize(df:pd.DataFrame, col_name, max_n_cat=-1, nans_to_zero=True):
    col=df[col_name]
    if not is_numeric_dtype(col) and len(col.cat.categories)> max_n_cat:
        df[col_name]=col.cat.codes
        if nans_to_zero:
            df[col_name]+=1

In [17]:
#export
def proc_df(df:pd.DataFrame, y_fld:str=None,
           na_dict:dict=None, skip_flds:list=None, 
           ignore_flds:list=None, max_n_cat: int=-1):
    '''
    returns df, y, na_dict
    '''
    if skip_flds is None: skip_flds=[]
    if ignore_flds is None: ignore_flds=[]
    if na_dict is None: na_dict={}
        
    assert sum(1 if not (is_categorical_dtype(df[col_name])) or
            is_numeric_dtype(df[col_name]) else 0
              for col_name in df.columns), 'all the columns in the df must be of type numerical or categorical'
    df = df.copy()
    ignored_cols = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    y=None
    if y_fld:
        if not is_numeric_dtype(df[y_fld]):
            df[y_fld]= df[y_fld].cat.codes
        y=df[y_fld].values
        skip_flds.append(y_fld)
        
    df.drop(skip_flds, axis=1, inplace=True)
    for col_name in df.columns:
        fill_missing(df, col_name, na_dict)
        numericalize(df, col_name, max_n_cat=max_n_cat)
        
    df = pd.get_dummies(df, dummy_na=True)
    df=pd.concat([ignored_cols, df], axis=1)
    return df, y, na_dict

In [18]:
#export
def add_datepart(df:pd.DataFrame, cols:list=None, time:bool=True,inplace:bool=True)->pd.DataFrame:
    '''parameters:
                df: pd.DataFrame
                cols: datetime cols
                just date: bool specifying whether the object is it just date or date time
                inplace: bool'''
    date_part=['year','month','day','week','dayofweek','weekday','quarter','is_month_start','is_month_end','is_year_end']
    time_part=['hour','minute','second']
    for i in cols:
        if time:
            for j in time_part:
                df[f'{i}_{j}']=getattr(df[i].dt,j)
        for j in date_part:
            df[f'{i}_{j}']=getattr(df[i].dt,j)
    df.drop(columns=cols, inplace=True)
    return 

In [19]:
add_datepart(df, ['timestamp'])

In [40]:
discrete = [var for var in df_train.columns if df_train[var].dtype!='O' and var!='price_doc' and df_train[var].nunique()<10]
continuous = [var for var in df_train.columns if df_train[var].dtype!='O' and var!='price_doc' and var not in discrete]
categorical = [var for var in df_train.columns if df_train[var].dtype=='O']

In [20]:
#export
def split_val(df:pd.DataFrame, val_pct:float=0.3):
    'returns `df_train` and `df_valid`'
    shuf_idx=np.random.permutation(len(df))
    train_idx, val_idx=shuf_idx[int(val_pct*len(df)):], shuf_idx[:int(val_pct*len(df))]
    return df.iloc[train_idx,:], df.iloc[val_idx,:]
    
def split_dep_col(df:pd.DataFrame, y:str):
    'returns `x` and `y`'
    return df.drop(columns=y), df.loc[:, y]
    
def train_test_split(df:pd.DataFrame, y:str=None, val_pct:float=0.3):
    ''' df 'pandas datadrame object `y` is the dependent column val_pct:=0.3'
    returns `x_train`, `y_train`, `x_valid`, `y_valid`'''
    df_train, df_val= split_val(df.copy(), val_pct)
    if y is None:
        return df_train, df_val
    else: return split_dep_col(df_train, y), split_dep_col(df_train, y)

In [21]:
df_train, df_val= train_test_split(df)

In [22]:
train_cats(df_train)
apply_cats(df_val, df_train)

In [23]:
x_train, y_train, nas=proc_df(df_train, 'price_doc')
x_val, y_val, nas= proc_df(df_val, 'price_doc', nas)

In [24]:
x_train.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_pr

In [21]:
from sklearn.ensemble import RandomForestRegressor

In [22]:
df.shape

(30471, 303)

In [59]:
regr=RandomForestRegressor(max_depth=6, max_samples=5000, n_jobs=-1, verbose=10, oob_score=True, min_samples_split=3)

In [73]:
param_grid={'n_estimators':list(range(100, 500, 100)),
            'max_depth':list(range(5, 15,2)), 
            'min_samples_split':[50, 25, 10] 
            }

In [24]:
regr=RandomForestRegressor(n_estimators=500,
                                        max_samples=0.5,
                                        n_jobs=-1,
                                        oob_score=True,
                                        max_depth=15)

In [25]:
regr.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=15, max_features='auto', max_leaf_nodes=None,
                      max_samples=0.5, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=-1, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [29]:
music_path=Path(r'e:\music\down\kuthu')
f'{list(music_path.iterdir())[11]}'

'e:\\music\\down\\kuthu\\OK Kanmani - Kaara Aattakkaara Video _ A.R. Rahman, Mani Ratnam-m-pK_pf-anc.mp3'

In [44]:
#export
# from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [31]:
mean_squared_error(regr.predict(x_val), y_val)

7342524103368.35

In [32]:
mean_squared_error(regr.predict(x_train), y_train)

3311326630517.6104

In [43]:
pd.DataFrame((np.arange(y_val.shape[0]),regr.predict(x_val), y_val))

0             1             2             3             4  \
0  0.000000e+00  1.000000e+00  2.000000e+00  3.000000e+00  4.000000e+00   
1  7.388391e+06  1.330733e+07  9.016884e+06  5.591895e+06  3.497528e+06   
2  7.950000e+06  1.449058e+07  2.500000e+06  5.036850e+06  3.347400e+06   

              5             6             7             8             9  \
0  5.000000e+00  6.000000e+00  7.000000e+00  8.000000e+00  9.000000e+00   
1  2.950383e+06  5.098611e+06  4.627575e+06  7.341161e+06  9.361893e+06   
2  3.205282e+06  9.990000e+05  4.709200e+06  7.419500e+06  2.000000e+06   

             10            11            12            13            14  \
0  1.000000e+01  1.100000e+01  1.200000e+01  1.300000e+01  1.400000e+01   
1  6.277381e+06  6.611719e+06  6.626179e+06  5.711672e+06  7.467870e+06   
2  1.000000e+06  6.524210e+06  7.800000e+06  6.500000e+06  8.474131e+06   

             15            16            17            18            19  \
0  1.500000e+01  1.600000e+01  1.700000e+01  1.800000e+01  1.900000e+01   
1  1.149744e+07  6.174073e+06  9.001877e+06  2.733450e+06  5.479303e+06   
2  1.100098e+07  6.150000e+06  2.000000e+06  3.004036e+06  5.062430e+06   

             20            21            22            23            24  \
0  2.000000e+01  2.100000e+01  2.200000e+01  2.300000e+01  2.400000e+01   
1  5.391080e+06  1.732068e+07  3.012746e+07  5.663270e+06  5.744114e+06   
2  5.500000e+06  9.900000e+05  6.000000e+07  6.250000e+06  6.976549e+06   

             25            26            27            28            29  \
0  2.500000e+01  2.600000e+01  2.700000e+01  2.800000e+01  2.900000e+01   
1  8.911749e+06  5.421399e+06  5.557600e+06  5.237970e+06  5.421747e+06   
2  1.009550e+07  5.800000e+06  5.202600e+06  5.992920e+06  5.638912e+06   

             30            31            32            33            34  \
0  3.000000e+01  3.100000e+01  3.200000e+01  3.300000e+01  3.400000e+01   
1  4.527379e+06  9.831397e+06  1.338211e+07  3.389534e+06  8.774729e+06   
2  4.127849e+06  1.028265e+07  1.643070e+07  5.900000e+06  2.200000e+06   

             35            36            37            38            39  \
0  3.500000e+01  3.600000e+01  3.700000e+01  3.800000e+01  3.900000e+01   
1  3.999584e+06  9.971700e+06  1.637472e+07  5.344026e+06  5.954391e+06   
2  4.333473e+06  1.030000e+07  1.580000e+07  7.000000e+06  5.156678e+06   

             40            41            42            43            44  \
0  4.000000e+01  4.100000e+01  4.200000e+01  4.300000e+01  4.400000e+01   
1  6.188234e+06  6.492485e+06  8.932190e+06  5.230110e+06  6.860231e+06   
2  6.240910e+06  7.900000e+06  9.000000e+06  6.600000e+06  8.200000e+06   

             45            46            47            48            49  \
0  4.500000e+01  4.600000e+01  4.700000e+01  4.800000e+01  4.900000e+01   
1  7.227332e+06  9.743389e+06  1.059587e+07  4.356083e+06  2.747477e+06   
2  7.300000e+06  9.650000e+06  1.200000e+07  5.600000e+06  3.000000e+06   

             50            51            52            53            54  \
0  5.000000e+01  5.100000e+01  5.200000e+01  5.300000e+01  5.400000e+01   
1  5.408335e+06  5.653284e+06  5.458115e+06  6.613768e+06  5.131390e+06   
2  5.292000e+06  5.300000e+06  6.579000e+06  8.000000e+06  5.500000e+06   

             55            56            57            58            59  \
0  5.500000e+01  5.600000e+01  5.700000e+01  5.800000e+01  5.900000e+01   
1  2.311406e+07  4.028720e+06  1.240677e+07  4.620871e+06  1.668896e+07   
2  1.000000e+07  4.016339e+06  1.600000e+07  5.100000e+06  1.980000e+07   

             60            61            62            63            64  \
0  6.000000e+01  6.100000e+01  6.200000e+01  6.300000e+01  6.400000e+01   
1  4.627930e+06  5.477217e+06  8.609923e+06  5.102172e+06  9.595216e+06   
2  3.573584e+06  5.200000e+06  8.425690e+06  5.280000e+06  1.455180e+07   

             65            66            67            68            69  \
0  6.500000e+01  6.600000e

In [41]:
y_val.shape

(9141,)

In [26]:
from catboost import CatBoostRegressor

In [27]:
model = CatBoostRegressor(iterations=500, 
                          depth=10, 
                          learning_rate=1, 
                          loss_function='RMSE',
                         verbose=50)

In [29]:
model.fit(
    x_train, y_train,
#     cat_features=categorical_features_indices,
    eval_set=(x_val, y_val),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 3101767.3888348	test: 3141794.5385134	best: 3141794.5385134 (0)	total: 806ms	remaining: 6m 42s
50:	learn: 1219725.1237143	test: 2873820.5517136	best: 2755403.5264839 (16)	total: 37.6s	remaining: 5m 30s
100:	learn: 733302.7464690	test: 2958908.6922889	best: 2755403.5264839 (16)	total: 1m 16s	remaining: 5m
150:	learn: 459985.4616728	test: 2980029.2140767	best: 2755403.5264839 (16)	total: 1m 55s	remaining: 4m 27s
200:	learn: 302895.9801704	test: 2984729.8754810	best: 2755403.5264839 (16)	total: 2m 33s	remaining: 3m 48s
250:	learn: 208103.4331385	test: 2992250.3059550	best: 2755403.5264839 (16)	total: 3m 11s	remaining: 3m 9s
300:	learn: 148462.7610277	test: 2992437.2371650	best: 2755403.5264839 (16)	total: 3m 49s	remaining: 2m 31s
350:	learn: 110393.2043567	test: 2993295.4114375	best: 2755403.5264839 (16)	total: 4m 27s	remaining: 1m 53s
400:	learn: 82399.5121176	test: 2993838.8614522	best: 2755403.5264839 (16)	total: 5m 5s	remaining: 1m 15s
450:	learn: 60774.2858444	test: 2993456